# Predictions of Crowdfunding Campaign Success with Machine Learning Approach

Over the past century there has been a dramatic increase in crowdfunding project activity, which offers an alternative for both creators and backers to sell products and invest in creative businesses respectively. However, empirical analysis shows that only one-third of crowdfunding campaigns could meet their fundraising goal. The aim of this project is to develop a model that predicts the success of crowdfunding project with machine learning approach. The datasets are retrospectively collected from Web Robots, Kickstarter website, and Indiegogo website. The model could provide insights in pre-lunching stage and in early stage of fundraising.

## Table of Contents
1. [Libraries](#libs)
2. [Reader](#reads)

## Libraries <a class="anchor" id="libs"></a>

In [ ]:
# standard libraries
import json

# 3rd party libraries
import pandas as pd

# custom libraries
from src.reader import *

## Reader <a class="anchor" id="reads"></a>

In [ ]:
path_json_file = "dataset/kickstarter-corpus.json"

df_ks = read_json(path_json_file)
display(df_ks.head())

---